In [1]:
from IPython.display import HTML

custom_css = """
<style>
/* Fondo general */
body {
    background-color: #95c1ed; /* azul muy claro */
    font-family: 'Segoe UI', Tahoma, sans-serif;
    color: #333333;
}

/* Títulos */
h1, h3, b {
    color: #333333;
}

/* Botones */
button {
    background-color: #1f7a9c !important;
    color: white !important;
    border-radius: 6px !important;
    padding: 6px 12px !important;
}
button:hover {
    background-color: #45a049 !important;
}

/* Widgets */
.widget-label {
    font-weight: bold;
    color: #444444;
}
.widget-dropdown, .widget-text {
    border-radius: 5px;
    border: 1px solid #ccc;
}

/* Tablas */
.dataframe {
    border-collapse: collapse;
    width: 100%;
}
.dataframe th, .dataframe td {
    border: 1px solid #ccc;
    padding: 6px;
}
.dataframe th {
    background-color: #e0e0e0;
}
</style>
"""
display(HTML(custom_css))


In [2]:
import pandas as pd
import io
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import seaborn as sns

In [3]:
from functools import lru_cache
import numpy as np

# Cachés en memoria para no recalcular todo cada vez
CACHE = {
    "summaries": {},        # (col) -> dict resumen
    "hist": {},             # (col, bins, log) -> (counts, edges)
    "value_counts": {}      # (col) -> Serie
}

def _key_hist(col, bins, log):
    return (col, int(bins), bool(log))


In [4]:
def infer_type(series):
    if pd.api.types.is_numeric_dtype(series):
        return "Numérica"
    else:
        return "Categórica"

#### Tabla con summary() #####

def summarize_variable(df, col):
    # Usa caché si ya lo calculamos
    if col in CACHE["summaries"]:
        return CACHE["summaries"][col]

    s = df[col]
    out = {
        "n": int(s.size),
        "n_null": int(s.isna().sum()),
        "n_unique": int(s.nunique(dropna=True))
    }

    if pd.api.types.is_numeric_dtype(s):
        x = pd.to_numeric(s, errors="coerce").dropna()
        if x.empty:
            out.update({"mean": None, "std": None, "min": None, "q1": None, "median": None, "q3": None, "max": None})
        else:
            q = x.quantile([0.25, 0.5, 0.75])
            out.update({
                "mean": float(x.mean()),
                "std": float(x.std(ddof=1)) if x.size > 1 else 0.0,
                "min": float(x.min()),
                "q1": float(q.loc[0.25]),
                "median": float(q.loc[0.5]),
                "q3": float(q.loc[0.75]),
                "max": float(x.max())
            })
    else:
        # value_counts cacheado para categóricas
        if col not in CACHE["value_counts"]:
            CACHE["value_counts"][col] = s.value_counts(dropna=False, sort=True)
        vc = CACHE["value_counts"][col]
        out["top5"] = vc.head(5)

    CACHE["summaries"][col] = out
    return out


In [5]:
##### Carguemos el excel #####

def leer_excel_flexible(raw_bytes):
    # Lee .xlsx con openpyxl (más estable en Render)
    df = pd.read_excel(io.BytesIO(raw_bytes), engine="openpyxl")

    # Opcional: convierte textos de baja cardinalidad a 'category' para ahorrar memoria
    for c in df.columns:
        s = df[c]
        if s.dtype == object and s.nunique(dropna=True) <= 1000:
            df[c] = s.astype("category")
    return df


def extract_content(upload_value):
    if isinstance(upload_value, dict):
        return list(upload_value.values())[0]
    elif isinstance(upload_value, (tuple, list)) and upload_value:
        return upload_value[0]
    else:
        raise ValueError("Formato inesperado de upload.value")

In [6]:
##### Interpretemos los resultados #####
def interpretar_variable_html(df, col, assigned_type):
    serie = df[col].dropna()
    n = len(serie)
    if n == 0:
        return "<p>No hay datos</p>"
    
    if "Categórica" in assigned_type:
        counts = serie.astype(str).value_counts(normalize=True)
        top_cat = counts.idxmax()
        top_pct = counts.max() * 100
        n_unique = serie.nunique()
        text = f"<p>La variable '<b>{col}</b>' es categórica con <b>{n_unique}</b> categorías únicas.</p>"
        text += f"<p>La categoría con más frecuencia es '<b>{top_cat}</b>', representando el <b>{top_pct:.1f}%</b> de los datos.</p>"
        return text
    
    # Variables numéricas
    mean = serie.mean()
    median = serie.median()
    q1 = serie.quantile(0.25)
    q3 = serie.quantile(0.75)
    iqr = q3 - q1
    std = serie.std()
    min_v = serie.min()
    max_v = serie.max()

    text = f"<p>La variable '<b>{col}</b>' es numérica con <b>{n}</b> observaciones.</p>"
    text += f"<p>Media (promedio) = <b>{mean:.3f}</b>, Mediana (Q2) = <b>{median:.3f}</b>.</p>"
    text += f"<p>La desviación estándar es <b>{std:.3f}</b>, lo que nos indica la distancia promedio de los datos a la media.</p>"
    text += f"<p>El rango intercuartílico IQR (Q3 - Q1) es <b>{iqr:.3f}</b>, lo que indica dispersión central.</p>"
    text += (
        "<ul>"
        "<li>Un IQR pequeño → el 50% central de los datos está muy agrupado (baja variabilidad central).</li>"
        "<li>Un IQR grande → el 50% central está muy disperso (alta variabilidad central).</li>"
        "</ul>"
    )
    text += f"<p>El rango total va de <b>{min_v:.3f}</b> a <b>{max_v:.3f}</b>.</p>"

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = serie[(serie < lower_bound) | (serie > upper_bound)]
    if len(outliers) > 0:
        text += f"<p>Se tienen <b>{len(outliers)}</b> valores atípicos (valores fuera de los límites) [{lower_bound:.3f}, {upper_bound:.3f}].</p>"
    else:
        text += "<p>No se detectaron valores atípicos evidentes.</p>"

    return text
    
# --- estado ---
state = {"df": None, "types": {}}

In [7]:
##### widgets #####

# --- Widgets mejorados ---

# Upload con icono y estilo
upload = widgets.FileUpload(
    accept='.xls,.xlsx',
    multiple=False,
    description="Sube tu archivo Excel",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='250px')
)

# Contenedor para selección de tipos de variables con estilo pastel y borde redondeado
type_override_box = widgets.VBox(
    layout=widgets.Layout(
        border='2px solid #c8d6e5',
        padding='10px',
        border_radius='10px',
        width='500px',
        overflow='auto',
        max_height='300px'
    )
)

# Output widgets
summary_out = widgets.Output(layout=widgets.Layout(border='1px solid #c8d6e5', padding='10px', border_radius='10px'))
plot_out = widgets.Output(layout=widgets.Layout(border='1px solid #c8d6e5', padding='10px', border_radius='10px'))
status_out = widgets.Output(layout=widgets.Layout(border='1px solid #c8d6e5', padding='10px', border_radius='10px'))

# Selector de variable con ancho definido
var_selector = widgets.Dropdown(
    description="Variable:",
    options=[],
    layout=widgets.Layout(width="400px"),
    style={'description_width': 'initial'}
)

# Slider de bins con color pastel y ancho definido
bins_slider = widgets.IntSlider(
    value=25,
    min=5,
    max=100,
    step=5,
    description="Bins:",
    layout=widgets.Layout(width="300px"),
    style={'description_width': '50px'}
)

# Checkbox para escala logarítmica
log_scale_chk = widgets.Checkbox(
    value=False,
    description="Escala log",
    indent=False,
    style={'description_width': 'initial'}
)



In [8]:
##### Tipos de variables #####
def build_type_override_widgets():
    children = []
    for col, inferred_type in state["types"].items():
        dd = widgets.Dropdown(
            options=["Categórica", "Numérica"],  # Solo estas opciones
            value=inferred_type,
            description=col,
            layout=widgets.Layout(width="450px")
        )
        def on_change(change, column=col):
            state["types"][column] = change["new"]
            update_summary()
            refresh_variable_selector()
            plot_variable()
        dd.observe(on_change, names="value")
        children.append(dd)
    type_override_box.children = children

In [9]:
def update_summary(*args):
    summary_out.clear_output()
    if state["df"] is None:
        return
    df = state["df"]
    
    with summary_out:
        clear_output()
        print("Resumen de variables")
        summary_list = [summarize_variable(df, col, state["types"][col]) for col in df.columns]
        summary_df = pd.DataFrame(summary_list)
        cols_order = ["Nombre","Tipo de variable","Vacios","Únicos","Categorías",
                      "Media","Mediana","Q1","Q3","Desviación","Min","Max","Error"]
        # Aseguramos que si no hay "Error" la columna exista
        if "Error" not in summary_df.columns:
            summary_df["Error"] = pd.NA
        summary_df = summary_df[cols_order]
        pd.options.display.float_format = '{:.3f}'.format
        display(summary_df)

        var = var_selector.value
        if var is not None:
            assigned_type = state["types"].get(var, "Categórica")
            display(HTML("<h4>Resultados:</h4>"))
            try:
                texto_interpretacion = interpretar_variable_html(df, var, assigned_type)
            except Exception as e:
                texto_interpretacion = f"<p style='color:red;'>❌ No se pudo generar interpretación: {repr(e)}</p>"
            display(HTML(texto_interpretacion))

            
def refresh_variable_selector():
    if state["df"] is None:
        var_selector.options = []
        return
    var_selector.options = list(state["df"].columns)
    if var_selector.options and var_selector.value not in var_selector.options:
        var_selector.value = var_selector.options[0]

In [10]:
import gc
##### Graficos #####
# --- helpers numéricas ---
def fast_hist(serie, bins=30, log=False):
    x = pd.to_numeric(serie, errors="coerce").dropna()
    if x.size > 200_000:  # limita para no congelar
        x = x.sample(200_000, random_state=0)
    if log:
        x = x[x > 0]
        x = np.log10(x)
    counts, edges = np.histogram(x, bins=bins)
    return counts, edges

def _kde_gauss(x, points=200, bw="scott"):
    """KDE simple sin SciPy (aprox). Si n>5000, samplea para rendimiento."""
    x = pd.to_numeric(x, errors="coerce").dropna().to_numpy()
    if x.size == 0:
        return None, None
    n = x.size
    if n > 5000:
        rng = np.random.default_rng(0)
        x = rng.choice(x, 5000, replace=False)
        n = x.size
    std = x.std(ddof=1) if n > 1 else 1.0
    if std == 0:
        return None, None
    if bw == "scott":
        h = (n ** (-1/5)) * std
    elif isinstance(bw, (int, float)):
        h = float(bw)
    else:
        h = (n ** (-1/5)) * std
    xs = np.linspace(x.min(), x.max(), points)
    diff = (xs[:, None] - x[None, :]) / h
    dens = np.exp(-0.5 * diff**2).sum(axis=1) / (np.sqrt(2*np.pi) * h * n)
    return xs, dens

# --- helpers categóricas ---
def _value_counts_cached(df, col):
    if col not in CACHE["value_counts"]:
        CACHE["value_counts"][col] = df[col].value_counts(dropna=False, sort=True)
    return CACHE["value_counts"][col]

# --- función principal ---
def plot_variable(df, col, kind="auto", bins=30, log=False, top_n=10, normalize=False, kde_bw="scott"):
    """
    kind:
      - 'auto'       → detecta tipo y elige: hist (numérico) o barras (categórico)
      - 'hist'       → histograma (numérico)
      - 'density'    → curva de densidad (numérico, KDE aproximada)
      - 'box'        → boxplot (numérico)
      - 'bar'        → barras (categórico)
      - 'pie'        → pastel (categórico)
    """
    s = df[col]
    is_num = pd.api.types.is_numeric_dtype(s)
    chosen = kind

    if chosen == "auto":
        chosen = "hist" if is_num else "bar"

    # --- numéricos ---
    if chosen in {"hist", "density", "box"}:
        if not is_num:
            print(f"La variable '{col}' no es numérica; usa kind='bar' o 'pie'.")
            return

        if chosen == "hist":
            key = _key_hist(col, bins, log)
            if key in CACHE["hist"]:
                counts, edges = CACHE["hist"][key]
            else:
                counts, edges = fast_hist(s, bins=bins, log=log)
                CACHE["hist"][key] = (counts, edges)

            plt.figure()
            centers = (edges[:-1] + edges[1:]) / 2.0
            width = (edges[1:] - edges[:-1])
            plt.bar(centers, counts, width=width)
            plt.xlabel(f"{col}" + (" (log10)" if log else ""))
            plt.ylabel("Frecuencia")
            plt.tight_layout()
            plt.show()
            plt.close('all')
            import gc; gc.collect()
            return

        if chosen == "density":
            xs, dens = _kde_gauss(s, points=256, bw=kde_bw)
            if xs is None:
                print("No hay suficientes datos numéricos para densidad.")
                return
            plt.figure()
            plt.plot(xs, dens)
            plt.xlabel(col)
            plt.ylabel("Densidad")
            plt.tight_layout()
            plt.show()
            plt.close('all')
            import gc; gc.collect()
            return

        if chosen == "box":
            x = pd.to_numeric(s, errors="coerce").dropna()
            if x.empty:
                print("No hay datos numéricos para boxplot.")
                return
            plt.figure()
            plt.boxplot(x, vert=True, showfliers=False)
            plt.ylabel(col)
            plt.tight_layout()
            plt.show()
            plt.close('all')
            import gc; gc.collect()
            return

    # --- categóricas ---
    if chosen in {"bar", "pie"}:
        vc = _value_counts_cached(df, col)
        if vc.empty:
            print("No hay categorías para graficar.")
            return
        vc = vc.head(int(top_n))
        if normalize:
            total = vc.sum()
            vals = (vc / total) * 100.0
        else:
            vals = vc

        if chosen == "bar":
            plt.figure()
            plt.bar(vc.index.astype(str), vals.values)
            plt.xticks(rotation=45, ha="right")
            plt.ylabel("Porcentaje (%)" if normalize else "Frecuencia")
            plt.xlabel(col)
            plt.tight_layout()
            plt.show()
            plt.close('all')
            import gc; gc.collect()
            return

        if chosen == "pie":
            plt.figure()
            autopct = (lambda p: f"{p:.1f}%") if normalize else (lambda p: f"{p:.0f}")
            plt.pie(vals.values, labels=vc.index.astype(str), autopct=autopct, startangle=90)
            plt.tight_layout()
            plt.show()
            plt.close('all')
            import gc; gc.collect()
            return

    print(f"kind='{kind}' no reconocido. Usa 'hist', 'density', 'box', 'bar' o 'pie'.")


In [11]:
def on_upload(change):
    plot_out.clear_output()
    summary_out.clear_output()
    status_out.clear_output()
    if not upload.value:
        with status_out:
            clear_output()
            print("No se ha subido ningún archivo.")
        return

    with status_out:
        clear_output()
        print("Procesando archivo...")

    try:
        content = extract_content(upload.value)
    except Exception as e:
        with status_out:
            clear_output()
            print("Error extrayendo el contenido del upload:", repr(e))
        return

    try:
        raw = content.get("content", None)
        if raw is None:
            raise KeyError("No se encontró 'content' en el objeto recibido.")
    except Exception as e:
        with status_out:
            clear_output()
            print("Error accediendo al contenido bruto del upload:", repr(e))
        return

    try:
        df, _ = leer_excel_flexible(raw)
        with status_out:
            clear_output()
            print(f"Archivo cargado correctamente.")
    except Exception as e:
        with status_out:
            clear_output()
            print("Error al leer el archivo Excel. Detalle completo:")
            import traceback
            traceback.print_exc()
        with summary_out:
            clear_output()
            print("No se pudo cargar el archivo. Revisa su formato.")
        return

    state["df"] = df
    inferred = {col: infer_type(df[col]) for col in df.columns}
    state["types"] = inferred.copy()

    build_type_override_widgets()
    update_summary()
    refresh_variable_selector()
    plot_variable()

In [12]:
##### Conexiones #####
upload.observe(on_upload, names="value")
var_selector.observe(update_summary, names="value")
var_selector.observe(plot_variable, names="value")
bins_slider.observe(plot_variable, names="value")
log_scale_chk.observe(plot_variable, names="value")

In [13]:
##### Edicion en html ####
display(widgets.HTML("""
<style>
@keyframes gradientMove {
    0% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
    100% { background-position: 0% 50%; }
}
</style>

<h1 style="
    color:black; 
    background: linear-gradient(270deg, #a8edea, #fed6e3, #cfd9df, #e0c3fc);
    background-size: 600% 600%;
    animation: gradientMove 8s ease infinite;
    padding:12px; 
    border-radius:10px; 
    text-align:center; 
    font-family:Arial; 
    font-size:28px;">
Estadística descriptiva
</h1>
"""))

display(widgets.HTML("""
<p>
El uso de la estadística descriptiva es fundamental para <b>presentar información de manera resumida e interpretar 
los resultados obtenidos</b> de un conjunto de datos.
</p>

<p>
La estadística descriptiva consiste en un <b>conjunto de técnicas para describir, resumir y analizar</b> la información de un conjunto de datos 🤓☝️. 
</p>

<p>
Vamos a trabajar con diferentes tipos de datos. Comenzaremos con una clasificación básica: <b>cuantitativos y cualitativos</b>.
</p>
"""))

display(widgets.HTML("""
<h3 style='color: #576574; text-align: left;'>🏷️ Tipos de variables</h3>
<div style='border: 2px solid #c8d6e5; border-radius: 10px; padding: 15px; 
            background-color: #f1f2f6; width: 1000px; margin: auto;'>
  <ul style='list-style: none; padding-left: 20px; margin: 0;'>
    <li>➤ <b>Variables cuantitativas</b>: son características que podemos medir y representar con números, generalmente corresponden a conteos o mediciones.</li>
    <li>➤ <b>Variables cualitativas</b>: son características relacionadas con cualidades, se dividen en diferentes categorías que por lo general no son numéricas.</li>
  </ul>
</div>
"""))

display(widgets.HTML("""
<h3 style='color: #576574; text-align: left;'>📈 Medidas descriptivas para variables numéricas</h3>
<div style='border: 2px solid #c8d6e5; border-radius: 10px; padding: 15px; 
            background-color: #f1f2f6; width: 1100px; margin: auto;'>
  <ul style='list-style: none; padding-left: 20px; margin: 0;'>
    <li>➤ <b>Media (aritmética)</b>: también conocida como promedio, consiste en sumar todos los elementos y dividir entre su número total. Es decir, realizar la suma de n valores y dividir entre n.
    <li>➤ <b>Mediana</b>: es el valor intermedio de un conjunto de datos cuando están ordenados de manera ascendente. Probabilísticamente, corresponde al valor que acumula 0.5 de probabilidad.</li>
    <li>➤ <b>Desviación estándar</b>: mide la variación de los datos respecto a la media.</li>
    <li>➤ <b>Q1</b>: acumula el 25 % de los datos; al menos el 25 % de los valores son menores o iguales a Q1.</li>
    <li>➤ <b>Q3</b>: acumula el 75 % de los datos; al menos el 75 % de los valores son menores o iguales a Q3.</li>
  </ul>
</div>
"""))


display(widgets.HTML("""
<h3 style='color: #576574; text-align: left;'>📊 Medidas descriptivas para variables categóricas</h3>
<div style='border: 2px solid #c8d6e5; border-radius: 10px; padding: 15px; 
            background-color: #f1f2f6; width: 1100px; margin: auto;'>
  <ul style='list-style: none; padding-left: 20px; margin: 0;'>
    <li>➤ <b>Valores únicos</b>: categorías existentes en la variable.</li>
    <li>➤ <b>Gráfico de barras</b>: representación gráfica de la frecuencia de las categorías de la variable.</li>
    <li>➤ <b>Gráfico de pay</b>: representación gráfica de la frecuencia de las categorías de la variable en términos porcentuales.</li>
  </ul>
</div>
"""))

display(widgets.HTML("""
<style>
@keyframes underlineGradient {
    0% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
    100% { background-position: 0% 50%; }
}
.animated-underline {
    display: inline-block;
    position: relative;
    font-size: 16px;
    font-weight: bold;
    color: #2f3542; 
    text-align: center;
}
.animated-underline::after {
    content: '';
    display: block;
    height: 5px;
    width: 100%;
    border-radius: 5px;
    background: linear-gradient(270deg, #a8edea, #fed6e3, #cfd9df, #e0c3fc); /* Colores pastel */
    background-size: 600% 600%;
    animation: underlineGradient 6s ease infinite;
    margin-top: 4px;
}
.centered-title {
    text-align: center;
    margin-bottom: 20px;
}
</style>

<div class="centered-title">
    <h2 class="animated-underline">🔧  A continuación te presentamos una herramienta que nos ayudará a conocer los datos</h2>
</div>
"""))

display(widgets.HTML("""
<h3 style='color: #576574;'>📄 Requisitos del archivo de Excel</h3>
<p>Necesitamos un archivo de Excel que contenga todas las variables que deseas analizar en <b>una sola hoja</b>.</p>
<p>Antes de subirlo, asegúrate de que:</p>
<ul style='list-style-type: disc; padding-left: 25px;'>
    <li>Las variables estén correctamente <b>limpias y clasificadas</b>.</li>
    <li>Las <b>variables numéricas</b> contengan únicamente números válidos.</li>
    <li>Las <b>variables categóricas</b> tengan cada categoría escrita de forma consistente y correcta.</li>
</ul>
<p>Esto garantizará un análisis preciso y libre de errores.</p>
"""))

display(widgets.HTML("""
<h3 style='color: #576574;'>📝  Paso a paso para analizar tus datos</h3>
"""))

display(widgets.HTML("""
<h4> 📤  1. Sube tu archivo Excel: Asegúrate de que tenga las características antes mencionadas.</h4>

<h4>
A continuación te comparto un archivo para que puedas probar esta herramienta.
Te lo dejo 👉 <a href='https://docs.google.com/spreadsheets/d/e/2PACX-1vSkMlWrodM_0UldxzyODQTuR8VqUrIpgR7PBKMSUXQRbqQnUtrfd7wGJWJsGmEqeA/pub?output=xlsx' 
   target='_blank' style='color: steelblue; text-decoration: underline; font-weight:bold;'>aquí</a>. Para una mejor compresion y correcto uso te dejo una 
   guia de los resultados que podras obtener con este conjunto de datos.
</h4>
"""))



display(upload)
display(status_out)
display(widgets.HTML("<h4> ✏️  2. Indica los tipos de variables:</h4>"))
display(type_override_box)
display(summary_out)

display(widgets.HTML("<h3>🔍  3. Visualiza el comportamiento de la variable</h3>"))
display(widgets.HBox([var_selector, bins_slider, log_scale_chk]))
display(plot_out)


HTML(value='\n<style>\n@keyframes gradientMove {\n    0% { background-position: 0% 50%; }\n    50% { backgroun…

HTML(value='\n<p>\nEl uso de la estadística descriptiva es fundamental para <b>presentar información de manera…

HTML(value="\n<h3 style='color: #576574; text-align: left;'>🏷️ Tipos de variables</h3>\n<div style='border: 2p…

HTML(value="\n<h3 style='color: #576574; text-align: left;'>📈 Medidas descriptivas para variables numéricas</h…

HTML(value="\n<h3 style='color: #576574; text-align: left;'>📊 Medidas descriptivas para variables categóricas<…

...

HTML(value='<h4> ✏️  2. Indica los tipos de variables:</h4>')

VBox(layout=Layout(border_bottom='2px solid #c8d6e5', border_left='2px solid #c8d6e5', border_right='2px solid…

Output(layout=Layout(border_bottom='1px solid #c8d6e5', border_left='1px solid #c8d6e5', border_right='1px sol…

HTML(value='<h3>🔍  3. Visualiza el comportamiento de la variable</h3>')

Output(layout=Layout(border_bottom='1px solid #c8d6e5', border_left='1px solid #c8d6e5', border_right='1px sol…